# Lab 3 - Spark MLlib

#####"A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E"
-Tom M. Mitchell

####Machine Learning - the science of getting computers to act without being explicitly programmed

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. It consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering (this example!), dimensionality reduction, as well as lower-level optimization primitives and higher-level pipeline APIs.

It divides into two packages:
- spark.mllib contains the original API built on top of RDDs.
- spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.


Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.

http://spark.apache.org/docs/latest/mllib-guide.html

##Online Purchase Recommendations

Learn how to create a recommendation engine using the Alternating Least Squares algorithm in Spark's machine learning library

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/ALS.png' width="70%" height="70%"></img>

###The data

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.  The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

http://archive.ics.uci.edu/ml/datasets/Online+Retail

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/FullFile.png' width="80%" height="80%"></img>

##Create an RDD from the csv data 

###Download the data

In [17]:
!rm 'OnlineRetail.csv.gz' -f
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz

--2016-04-22 20:50:23--  https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 23.235.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|23.235.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7483128 (7.1M) [application/octet-stream]
Saving to: 'OnlineRetail.csv.gz'

100%[======================================>] 7,483,128   22.3MB/s   in 0.3s   

2016-04-22 20:50:24 (22.3 MB/s) - 'OnlineRetail.csv.gz' saved [7483128/7483128]



###Put the csv into an RDD (at first, each row in the RDD is a string which correlates to a line in the csv)

In [18]:
loadRetailData = sc.textFile("OnlineRetail.csv.gz")
print loadRetailData.take(2)

[u'InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country', u'536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom']


##Prepare and shape the data:  "80% of a Data Scientists  job"

####Remove the header from the RDD and split the string in each row by comma

In [19]:
header = loadRetailData.first()
loadRetailData = loadRetailData.filter(lambda line: line != header).\
                            map(lambda l: l.split(","))

print loadRetailData.take(2)

[[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom'], [u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']]


#####NOTE:  The original file at UCI's Machine Learning Repository has commas in the product description.  Those have been removed to expediate the lab.
####Only keep rows that have a purchase quantity of greater than 0, a customerID not equal to 0, and a non blank stock code after removing non-numeric characters.

In [20]:
import re

loadRetailData = loadRetailData.filter(lambda l: int(l[3]) > 0\
                                and len(re.sub("\D", "", l[1])) != 0 \
                                and len(l[6]) != 0)

print loadRetailData.take(2)

[[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom'], [u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']]


####Map each line to a row and create a data frame 

In [21]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

#Convert each line to a Row.
loadRetailData = loadRetailData.map(lambda l: Row(inv=int(l[0]),\
                                    stockCode=int(re.sub("\D", "", l[1])),\
                                    description=l[2],\
                                    quant=int(l[3]),\
                                    invDate=l[4],\
                                    price=float(l[5]),\
                                    custId=int(l[6]),\
                                    country=l[7]))

# Infer the schema, and register the DataFrame as a table.
retailDf = sqlContext.createDataFrame(loadRetailData)
print retailDf.printSchema()

retailDf.registerTempTable("retailPurchases")
print sqlContext.sql("SELECT * FROM retailPurchases limit 2").toPandas()

root
 |-- country: string (nullable = true)
 |-- custId: long (nullable = true)
 |-- description: string (nullable = true)
 |-- inv: long (nullable = true)
 |-- invDate: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quant: long (nullable = true)
 |-- stockCode: long (nullable = true)

None
          country  custId                         description     inv  \
0  United Kingdom   17850  WHITE HANGING HEART T-LIGHT HOLDER  536365   
1  United Kingdom   17850                 WHITE METAL LANTERN  536365   

        invDate  price  quant  stockCode  
0  12/1/10 8:26   2.55      6      85123  
1  12/1/10 8:26   3.39      6      71053  


####Keep only the data we need (custId, stockCode, and rank)

In [22]:
query = """
SELECT 
    custId, stockCode, 1 as purch
FROM 
    retailPurchases 
group 
    by custId, stockCode"""
retailDf = sqlContext.sql(query)

print retailDf.take(2)

[Row(custId=13534, stockCode=23389, purch=1), Row(custId=13029, stockCode=84874, purch=1)]


###Randomly split the data into a testing set (10% of the data), a cross validation set (10% of the data) a training set (80% of the data)

In [23]:
testDf, cvDf, trainDf = retailDf.randomSplit([.1,.1,.8],1)

print trainDf.take(2)
print cvDf.take(2)
print testDf.take(2)

[Row(custId=13534, stockCode=23389, purch=1), Row(custId=13029, stockCode=84874, purch=1)]
[Row(custId=12646, stockCode=23245, purch=1), Row(custId=15235, stockCode=22652, purch=1)]
[Row(custId=14426, stockCode=85185, purch=1), Row(custId=14499, stockCode=82484, purch=1)]


##Build recommendation models

####Use training DF to train a model with Alternating Least Squares 
Latent Factors / rank<br>
The number of columns in the user-feature and product-feature matricies)<br>
Iterations / maxIter<br>
The number of factorization runs<br>

In [24]:
from pyspark.ml.recommendation import ALS

als1 = ALS(rank=15, maxIter=5,userCol="custId",itemCol="stockCode",ratingCol="purch")
model1 = als1.fit(trainDf)

als2 = ALS(rank=2, maxIter=10,userCol="custId",itemCol="stockCode",ratingCol="purch")
model2 = als2.fit(trainDf)

print "The models has been trained"

The models has been trained


##Test the models

Use the models to predict what the user will rate a certain item.  The closer our model to 1 that our model rates an item a user has already purchased, the better.

####Evaluate the model with the cross validation dataframe by using the transform function.

Some of the users or purchases in the cross validation data may not have been in the training data.  Let's remove the ones that are not.

In [30]:
customers = set(trainDf.map(lambda line: line.custId).collect())
stock = set(trainDf.map(lambda line: line.stockCode).collect())

print cvDf.count()
cvDf = cvDf.rdd.filter(lambda line: line.stockCode in stock and\
                                           line.custId in customers).toDF()
print cvDf.count()

26231
26231


In [31]:
predictions1 = model1.transform(cvDf)
predictions2 = model2.transform(cvDf)

print predictions1.take(2)
print predictions2.take(2)

[Row(custId=14247, stockCode=71038, purch=1, prediction=0.900556206703186), Row(custId=12447, stockCode=21238, purch=1, prediction=0.9004534482955933)]
[Row(custId=15861, stockCode=23235, purch=1, prediction=0.9010478258132935), Row(custId=14081, stockCode=85035, purch=1, prediction=0.9010478258132935)]


####Calculate and print the Mean Squared Error.   For all ratings, subtract the prediction from the actual purchase (1), square the result, and take the mean of all of the squared differences.

In [32]:
meanSquaredError1 = predictions1.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError2 = predictions2.map(lambda line: (line.purch - line.prediction)**2).mean()
    
print 'Mean squared error = %.4f for our first model' % meanSquaredError1
print 'Mean squared error = %.4f for our second model' % meanSquaredError2

Mean squared error = 0.0099 for our first model
Mean squared error = 0.0098 for our second model


####Confirm the model by testing it with the test data and the best hyperparameters found during cross validation

In [33]:
filteredTestDf = testDf.rdd.filter(lambda line: line.stockCode in stock and\
                                              line.custId in customers).toDF()
predictions3 = model2.transform(filteredTestDf)
meanSquaredError3 = predictions3.map(lambda line: (line.purch - line.prediction)**2).mean()
    
print 'Mean squared error = %.4f for our best model' % meanSquaredError3

Mean squared error = 0.0098 for our best model


#Implement the model

Use the best model to predict items the user will be interested in.

First, create a dataframe in which each row has the user id and an item id.

In [34]:
from pyspark.sql.functions import lit

userItems = trainDf.select("stockCode").distinct().\
            withColumn('custId', lit(15544))

print userItems.take(5)

[Row(stockCode=22631, custId=15544), Row(stockCode=20831, custId=15544), Row(stockCode=23431, custId=15544), Row(stockCode=23031, custId=15544), Row(stockCode=21631, custId=15544)]


Use 'transform' to rate each item.

In [35]:
userItems = model2.transform(userItems)

print userItems.take(5)

[Row(stockCode=84836, custId=15544, prediction=0.9010478258132935), Row(stockCode=44236, custId=15544, prediction=0.9010478258132935), Row(stockCode=22436, custId=15544, prediction=0.9010478854179382), Row(stockCode=23236, custId=15544, prediction=0.9010478258132935), Row(stockCode=90036, custId=15544, prediction=0.9010478258132935)]


Print the top 5 recommendations.

In [15]:
print userItems.sort("prediction",ascending=False).take(5)

[Row(stockCode=90012, custId=15544, prediction=0.9010831117630005), Row(stockCode=90173, custId=15544, prediction=0.9010481238365173), Row(stockCode=90000, custId=15544, prediction=0.9010480642318726), Row(stockCode=84857, custId=15544, prediction=0.9010480642318726), Row(stockCode=84920, custId=15544, prediction=0.9010480046272278)]


Let's look up this user and the recommended product ID's in the excel file...

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/user.png' width="80%" height="80%"></img>

This user seems to have purchased a lot of childrens gifts and some holiday items.  The recomendation engine we created suggested some items along these lines

#####The ALS algorithm uses some randomness, so the recommendations yours produces may be different than these.

In [16]:
query = """
SELECT 
    distinct description 
FROM 
    retailPurchases 
WHERE 
    stockCode in (90012,90173,90000,84857,84920)
"""
items = sqlContext.sql(query)
print items.toPandas()

                           description
0             PINK MONTE CARLO HANDBAG
1            PAIR BUTTERFLY HAIR CLIPS
2  MIDNIGHT BLUE COPPER FLOWER NECKLAC
3          BLACK DROP CRYSTAL NECKLACE
4             BLUE MONTE CARLO HANDBAG
5   COPPER/OLIVE GREEN FLOWER NECKLACE
6  MIDNIGHT BLUE DROP CRYSTAL NECKLACE
7              PINK FLOWER FABRIC PONY
8  RASPBERRY ANT COPPER FLOWER NECKLAC


#####Data Citation
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).